<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [1]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import keras

In [2]:
## Define custom metric(s)
@keras.saving.register_keras_serializable(package="custom_metrics", name="avg_m_diff")
def avg_m_diff(y_true, y_pred):
    # Ensure the input tensors have the correct shape
    assert y_true.shape == y_pred.shape
    assert y_true.shape[2] == 2
    
    def haversine_distance(lat1, lon1, lat2, lon2):
        # Radius of the Earth in meters
        R = 6371000.0
        
        # Convert latitude and longitude from degrees to radians
        lat1 = lat1 * (tf.constant(3.141592653589793) / 180.0)
        lon1 = lon1 * (tf.constant(3.141592653589793) / 180.0)
        lat2 = lat2 * (tf.constant(3.141592653589793) / 180.0)
        lon2 = lon2 * (tf.constant(3.141592653589793) / 180.0)
        
        # Compute differences
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        # Haversine formula
        a = tf.math.sin(dlat / 2)**2 + tf.math.cos(lat1) * tf.math.cos(lat2) * tf.math.sin(dlon / 2)**2
        c = 2 * tf.math.atan2(tf.math.sqrt(a), tf.math.sqrt(1 - a))
        
        # Distance in meters
        distance = R * c
        
        return distance
    
    # Reshape the tensors to 2D arrays for easier manipulation
    y_true_flat = tf.reshape(y_true, [-1, 2])
    y_pred_flat = tf.reshape(y_pred, [-1, 2])
    
    # Split the coordinates into separate tensors
    lat_true, lon_true = tf.split(y_true_flat, num_or_size_splits=2, axis=1)
    lat_pred, lon_pred = tf.split(y_pred_flat, num_or_size_splits=2, axis=1)
    
    # Calculate the distance for each pair of points
    distances = haversine_distance(lat_true, lon_true, lat_pred, lon_pred)
    
    # Calculate the average distance
    average_distance = tf.reduce_mean(distances)
    
    return average_distance

In [10]:
## Load the model
model_16 = tf.keras.models.load_model('model_16_lr001.keras')
model_32 = tf.keras.models.load_model('model_32_lr001.keras')
model_64 = tf.keras.models.load_model('model_64_lr001.keras')
model_128 = tf.keras.models.load_model('model_128_lr001.keras')
model_32_32 = tf.keras.models.load_model('model_32_32_lr001.keras')
model_64_64 = tf.keras.models.load_model('model_64_64_lr001.keras')
model_16_v2 = tf.keras.models.load_model('model_16_lr001_v2.keras')

models = [model_16, model_32, model_64, model_128, model_32_32, model_64_64, model_16_v2]
model_names = ["model_16", "model_32", "model_64", "model_128", "model_32_32", "model_64_64", "model_16_v2"]
model_colors = ["pink", "blue", "green", "yellow", "cyan", "purple", "orange"]

# Show the model architecture
model_16.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 60, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16)             │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           340 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 10, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_1 (Normalization) │ (None, 10, 2)          │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,680 (18.29 KB)

 Trainable params: 1,556 (6.08 KB)

 Non-trainable params: 10 (48.00 B)

 Optimizer params: 3,114 (12.17 KB)

In [11]:
## Load numpy test data
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [16]:
## ONE STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 273

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])

pred_coords = {}
x = 0
for i in models:
  pred_coords["pred_coords_{0}".format(model_names[x])] = i.predict(x_test[num].reshape(1, num_of_timesteps, 2))
  x += 1

true_coords = y_test[num]

fig = go.Figure()

## Map display of True Point vs Predicted Point
# Get route coordinates
past_lat = []
past_lon = []

for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))


# Get prediction coordinates
x = 0
for i in pred_coords:
  pred_lat = []
  pred_lon = []

  for step in pred_coords[i]:
    for point in step:
      pred_lat.append(point[0])
      pred_lon.append(point[1])

  fig.add_trace(go.Scattermapbox(
    mode="markers+lines",
    lat=pred_lat,
    lon=pred_lon,
    name=i,
    marker={'size': 10, 'color': model_colors[x]}))
  x += 1


# Get true coordinates from y
true_lat = []
true_lon = []

for point in true_coords:
  true_lat.append(point[0])
  true_lon.append(point[1])

print("True lat:")
print(true_lat)
fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=true_lat,
  lon=true_lon,
  name="True Point",
  marker={'size': 10, 'color': 'red'}))


fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
True lat:
[-6.1288533, -6.1290133, -6.1291667, -6.1293133, -6.1294717, -6.1296283, -6.12979, -6.1299333, -6.1300817, -6.1302283]
